In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# Create Spark and SQL contexts
sc = SparkContext('spark://headnodehost:7077', 'pyspark')
sqlContext = SQLContext(sc)

# Create an RDD from the data
storeText = sc.textFile("wasb://sparkcontain@richawecdb3h3.blob.core.windows.net/sparkfolder/528CommaRemoved.csv")

# create a schema
storeSchema = StructType([StructField("Order_ID", IntegerType(), False),StructField("Order_Date", StringType(), False),StructField("Order_Priority", StringType(), False),StructField("Order_Quantity", IntegerType(), False),StructField("Sales", DoubleType(), False),StructField("Discount", DoubleType(), False),StructField("Ship_Mode", StringType(), False),StructField("Profit", DoubleType(), False),StructField("Unit_Price", DoubleType(), False),StructField("Shipping_cost", DoubleType(), False),StructField("Customer_Name", StringType(), False),StructField("Province", StringType(), False),StructField("Region", StringType(), False),StructField("Customer_Segment", StringType(), False),StructField("Product_Category", StringType(), False),StructField("Product_SubCategory", StringType(), False),StructField("Product_Container", StringType(), False),StructField("Product_Base_Margin", StringType(), False),StructField("Ship_Date", StringType(), False)])

# Parse the data
store = storeText.map(lambda s: s.split(",")).filter(lambda s: s[0] != "Order_ID").map(lambda s:(int(s[0]), str(s[1]), str(s[2]), int(s[3]), float(s[4]), float(s[5]), str(s[6]), float(s[7]), float(s[8]), float(s[9]), str(s[10]), str(s[11]), str(s[12]), str(s[13]), str(s[14]), str(s[15]), str(s[17]), str(s[18]), str(s[19]) ))

# Create a data frame
storedf = sqlContext.createDataFrame(store,storeSchema)

# Register the data fram as a table to run queries against
storedf.registerAsTable("store")

In [2]:
# Sample Query - Run queries against the table and display the data
data = sqlContext.sql("select * from store limit 5")
data.show()

Order_ID Order_Date Order_Priority Order_Quantity Sales     Discount Ship_Mode      Profit  Unit_Price Shipping_cost Customer_Name      Province Region  Customer_Segment Product_Category Product_SubCategory  Product_Container Product_Base_Margin Ship_Date 
3        2010-10-13 Low            6              261.54    0.04     Regular Air    -213.25 38.94      35.0          Muhammed MacIntyre Nunavut  Nunavut Small Business   Office Supplies  Storage & Organiz... Large Box         0.8                 10/20/2010
293      2012-02-20 High           49             10123.02  0.07     Delivery Truck 457.81  208.16     68.02         Barry French       Nunavut  Nunavut Consumer         Office Supplies  Appliances           Jumbo Drum        0.58                10/2/2012 
293      2011-07-15 High           27             244.57    0.01     Regular Air    46.71   8.69       2.99          Barry French       Nunavut  Nunavut Consumer         Office Supplies  Binders and Binde... Small Box         0.3

In [3]:
# query of sales in province
sales_in_province = sqlContext.sql("select Province, sum(Sales) as sum_sales from store group by Province")
sales_in_province.show()

Province             sum_sales         
British Columbia     1892757.7845000005
Ontario              3063212.479499997 
Nova Scotia          817729.3624999998 
Yukon                975867.370999999  
Nunavut              116376.4835       
Manitoba             1372848.782       
Newfoundland         102924.06749999998
Saskachewan          1464455.8194999998
Prince Edward Island 409383.25000000023
Quebec               1510195.0799999996
New Brunswick        684211.5235000002 
Northwest Territo... 800847.3295000001 
Alberta              1704791.4910000006


In [4]:
# query for hoilday trend
hoilday = sqlContext.sql("select Order_Date, Sales from store")
hoilday.show()

Order_Date Sales    
2010-10-13 261.54   
2012-02-20 10123.02 
2011-07-15 244.57   
2011-07-15 4965.7595
2011-07-15 394.27   
2011-07-15 146.69   
2011-10-22 93.54    
2011-10-22 905.08   
2011-11-02 2781.82  
2011-03-17 228.41   
2009-01-19 196.85   
2009-06-03 124.56   
2009-06-03 716.84   
2010-12-17 1474.33  
2010-12-17 80.61    
2009-04-16 1815.49  
2010-01-28 248.26   
2012-11-18 4462.23  
2012-05-07 663.784  
2012-05-07 834.904  


In [5]:
# query for hoilday trend
hoilday = sqlContext.sql("select Order_Date, Sales from store")
hoilday.show()

Order_Date Sales    
2010-10-13 261.54   
2012-02-20 10123.02 
2011-07-15 244.57   
2011-07-15 4965.7595
2011-07-15 394.27   
2011-07-15 146.69   
2011-10-22 93.54    
2011-10-22 905.08   
2011-11-02 2781.82  
2011-03-17 228.41   
2009-01-19 196.85   
2009-06-03 124.56   
2009-06-03 716.84   
2010-12-17 1474.33  
2010-12-17 80.61    
2009-04-16 1815.49  
2010-01-28 248.26   
2012-11-18 4462.23  
2012-05-07 663.784  
2012-05-07 834.904  


In [6]:
#query of product_subcategory and sum of sales
sales_of_SubCatg = sqlContext.sql("select sum(Sales) as sum_sales, Product_SubCategory from store group by Product_SubCategory")
sales_of_SubCatg.show()

sum_sales          Product_SubCategory 
1130361.2999999998 Copiers and Fax     
80996.30999999998  Scissors Rulers a...
736991.5400000003  Appliances          
795875.9400000006  Computer Peripherals
1896008.142        Tables              
446452.8600000001  Paper               
1889313.8020000006 Telephones and Co...
822652.0400000003  Bookcases           
1022957.5900000003 Binders and Binde...
174085.80000000005 Envelopes           
167107.21999999997 Pens & Art Supplies 
38981.55000000001  Labels              
1070182.6000000003 Storage & Organiz...
2168697.14         Office Machines     
1761836.5500000003 Chairs & Chairmats  
15006.629999999997 Rubber Bands        
698093.81          Office Furnishings  


In [7]:
#query of customer_segment and sum of sales
sales_in_CustSeg = sqlContext.sql("select sum(Sales) as sum_sales, Customer_Segment from store group by Customer_Segment");
sales_in_CustSeg.show()

sum_sales         Customer_Segment
3564763.874999999 Home Office     
2788320.989499999 Small Business  
5498904.877500001 Corporate       
3063611.081999996 Consumer        
